In [1]:
sqlfile = r"ddl.sql"

with open(sqlfile, "r") as f:
    r = f.read()

In [2]:
r[:20]

'CREATE TABLE Adobe_A'

In [3]:
import pandas as pd

tabletable = pd.DataFrame(columns=["tablename", "column_name", "datatype", "nullable", "pk", "unique"])

Let's do tables first

In [4]:
table_names = []
for line in r.split("\n"):
    if line.startswith("CREATE TABLE"):
        current_table = line.split(" ")[2].replace("(", "")
        table_names.append(current_table)

groups = r.split(";")
print(groups[:5])

["CREATE TABLE Adobe_AdditionalMetadata (\n    id_local INTEGER PRIMARY KEY,\n    id_global UNIQUE NOT NULL,\n    additionalInfoSet INTEGER NOT NULL DEFAULT 0,\n    embeddedXmp INTEGER NOT NULL DEFAULT 0,\n    externalXmpIsDirty INTEGER NOT NULL DEFAULT 0,\n    image INTEGER,\n    incrementalWhiteBalance INTEGER NOT NULL DEFAULT 0,\n    internalXmpDigest,\n    isRawFile INTEGER NOT NULL DEFAULT 0,\n    lastSynchronizedHash,\n    lastSynchronizedTimestamp NOT NULL DEFAULT -63113817600,\n    metadataPresetID,\n    metadataVersion,\n    monochrome INTEGER NOT NULL DEFAULT 0,\n    xmp NOT NULL DEFAULT ''\n)", '\n\nCREATE INDEX index_Adobe_AdditionalMetadata_imageAndStatus ON Adobe_AdditionalMetadata( image, externalXmpIsDirty )', '\n\nCREATE TABLE Adobe_faceProperties (\n    id_local INTEGER PRIMARY KEY,\n    face INTEGER,\n    propertiesString\n)', '\n\nCREATE INDEX index_Adobe_faceProperties_face ON Adobe_faceProperties( face )', '\n\nCREATE TABLE Adobe_imageDevelopBeforeSettings (\n    

In [5]:
c = 0
max_table_name = max([len(x) for x in table_names])

def default_vals(row, row_split):
    default = True
    default_type = "None"
    default_val = None
    d_index = row_split.index("DEFAULT")
    odefault_val = row_split[d_index + 1]
    dint, dfloat = False, False
    try:
        default_val = int(odefault_val)
        dint = True
        default_type = "integer"
    except Exception as e: 
        dint = False
        #print(e)
    if dint == False:
        try:
            default_val = float(odefault_val)
            dfloat = True
            default_type = "float"
        except Exception as e:
            #print(e)
            dfloat = False
    if dfloat == False and dint == False:
        default_val = odefault_val
        default_type = "text"
        return default, default_type, default_val

def data_type(row):
    if "INTEGER" in row:
        data_type = "INTEGER"
    elif "TEXT" in row:
        data_type = "TEXT"
    elif "REAL" in row: 
        data_type = "FLOAT"
    elif "BLOB" in row:
        data_type = "BLOB"
    else:
        data_type = "NOT SPECIFIED"
    return(data_type)

for table in table_names:
    for group in groups:
        for line in group.split("\n"):
            if "CREATE TABLE" in line and table in line:
                rows = group.split("(")[1].split(")")[0]
                for row in rows.split(","):
                    nn = False
                    default = False
                    default_val = None
                    pk = False
                    unique = False
                    row = row.replace("\n", "").replace("\t", "")
                    row_split_boolean = False
                    row_split = row.split(" ")
                    while row_split_boolean == False:
                        if row_split[0] != "":
                            row_split_boolean = True
                        else:
                            row_split = row_split[1:]
                    column_name = row_split[0]
                    datatype = data_type(row)
                    if "PRIMARY KEY" in row:
                        pk = True
                    if "NOT NULL" in row:
                        nn = True
                    # if "DEFAULT" in row:
                    #     default, default_type, default_val = default_vals(row, row_split)
                    # else:
                    #     default, default_type, default_val = None, None, None
                    if "UNIQUE" in row:
                        unique = True
                    # ["tablename", "datatype", "nullable", "pk", "unique", "default", "d_int", "d_str"]
                    tabletable.loc[c] = [table, column_name, datatype, nn, pk, unique]
                    c += 1
                    # print(f"{column_name:^{max_table_name}} - {pk:5} - {nn:5} - {unique:5} - {str(default_val):^16} - {type(default_val)}")
                    

In [6]:
tabletable

,tablename,column_name,datatype,nullable,pk,unique
0,Adobe_AdditionalMetadata,id_local,INTEGER,False,True,False
1,Adobe_AdditionalMetadata,id_global,NOT SPECIFIED,True,False,True
2,Adobe_AdditionalMetadata,additionalInfoSet,INTEGER,True,False,False
3,Adobe_AdditionalMetadata,embeddedXmp,INTEGER,True,False,False
4,Adobe_AdditionalMetadata,externalXmpIsDirty,INTEGER,True,False,False
...,...,...,...,...,...,...
856,MigratedImages,UNIQUE,NOT SPECIFIED,False,False,True
857,MigratedInfo,ozCatalogId,TEXT,False,True,False
858,MigratedInfo,migrationStatus,NOT SPECIFIED,True,False,False
859,MigratedInfo,timestamp,NOT SPECIFIED,True,False,False


In [7]:
imports = """from base import Base

from sqlalchemy.orm import mapped_column, Mapped
from sqlalchemy import Text, Integer, Float, DateTime, Date, Time, Boolean, Uuid
from sqlalchemy import BLOB

from datetime import datetime"""

In [102]:
import pathlib
code_folder = "automated"

def col2rec(row):
    print(row)
    mc = []
    if row.pk == True:
        mc.append("primary_key=True")
    if row.nullable == True:
        mc.append("nullable=False")
    # else:
    #     mc.append("nullable=True")
    if row.unique == True:
        mc.append("unique=True")
    full_rec = f"{row.column_name}: Mapped[] = mapped_column({', '.join(mc)})"
    return(full_rec)

for table in table_names:
    outfile = pathlib.Path(code_folder) / f"{table}.py"
    with open(outfile, "w") as f:
        f.write(imports)
        f.write("\n\n")
        f.write(f"class {table}(Base):\n")
        f.write(f'\t__tablename__ = "{table}"\n')
        table_select = tabletable.loc[(tabletable['tablename'] == table)]
        for row in table_select.itertuples():
            f.write(f"\t{col2rec(row)}\n")
        



Pandas(Index=0, tablename='Adobe_AdditionalMetadata', column_name='id_local', datatype='INTEGER', nullable=False, pk=True, unique=False)
Pandas(Index=1, tablename='Adobe_AdditionalMetadata', column_name='id_global', datatype='NOT SPECIFIED', nullable=True, pk=False, unique=True)
Pandas(Index=2, tablename='Adobe_AdditionalMetadata', column_name='additionalInfoSet', datatype='INTEGER', nullable=True, pk=False, unique=False)
Pandas(Index=3, tablename='Adobe_AdditionalMetadata', column_name='embeddedXmp', datatype='INTEGER', nullable=True, pk=False, unique=False)
Pandas(Index=4, tablename='Adobe_AdditionalMetadata', column_name='externalXmpIsDirty', datatype='INTEGER', nullable=True, pk=False, unique=False)
Pandas(Index=5, tablename='Adobe_AdditionalMetadata', column_name='image', datatype='INTEGER', nullable=False, pk=False, unique=False)
Pandas(Index=6, tablename='Adobe_AdditionalMetadata', column_name='incrementalWhiteBalance', datatype='INTEGER', nullable=True, pk=False, unique=False)


In [15]:
import pathlib
table_csv = pathlib.Path("tabletable.csv")

tabletable.to_csv(table_csv, sep=";")